Render an image from a pose and 3DGS

In [12]:
import torch
print(torch.__version__)          # 2.9.0+cu128
print(torch.version.cuda)         # 12.8
print(torch.cuda.is_available())  # True if GPU is accessible

import numpy as np
from gsplat import rendering
from src.data_loading import SLAT_PLY_PATH
import os
from tracking_recovery_splat_alignment.splat_io.ply_reader import PlyReader
from PIL import Image

%load_ext autoreload
%autoreload 2

2.9.0+cu128
12.8
True
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
# load_dotenv()
# SRC_3DGS_SPLYS_DIR = os.getenv("SRC_3DGS_SPLYS_DIR")
LOCAL_DATA_PATH = os.path.expanduser("~/humanoid/temp_data/")
DIR = "PeachTest3-Asta-Splat"
SPLAT_NAME = "PeachTest3-Asta-Splat.ply"

Parse PLY file

In [15]:
# ply_path = os.path.join(SLAT_PLY_PATH, "Field and Construction Bay from Intel Screen Record - Attempt 2 without smart keyframing.ply")
ply_path = os.path.join(LOCAL_DATA_PATH, DIR, SPLAT_NAME)

In [16]:
ply_reader = PlyReader(ply_path)
vertex_data = ply_reader.read_vertex_array()

# Positions
means = torch.tensor(
    np.stack([vertex_data['x'], vertex_data['y'], vertex_data['z']], axis=-1),
    dtype=torch.float32
).to(device)

# Orientations (quaternions)
quats = torch.tensor(
    np.stack([vertex_data['rot_0'], vertex_data['rot_1'], vertex_data['rot_2'], vertex_data['rot_3']], axis=-1),
    dtype=torch.float32
).to(device)


# Scales
scales = torch.tensor(
    np.stack([vertex_data['scale_0'], vertex_data['scale_1'], vertex_data['scale_2']], axis=-1),
    dtype=torch.float32
).to(device)


# Opacities
opacities = torch.tensor(vertex_data['opacity'], dtype=torch.float32).to(device)

# Colors (normalize to 0..1)
colors = (torch.tensor(
    np.stack([vertex_data['f_dc_0'], vertex_data['f_dc_1'], vertex_data['f_dc_2']], axis=-1), dtype=torch.float32
) / 255.0).to(device)

In [ ]:
# -------------------------------
# Define camera (example: look at origin)
# -------------------------------
def look_at(eye, target, up):
    """Create a view matrix (right-handed, OpenGL style)."""
    f = (target - eye)
    f = f / np.linalg.norm(f)
    r = np.cross(up, f)
    r = r / np.linalg.norm(r)
    u = np.cross(f, r)
    view = np.eye(4, dtype=np.float32)
    view[:3, 0] = r
    view[:3, 1] = u
    view[:3, 2] = f
    view[:3, 3] = eye
    return torch.tensor(view)

eye = np.array([0.0, 0.0, -5.0])
target = np.array([0.0, 0.0, 0.0])
up = np.array([0.0, 1.0, 0.0])
viewmat = look_at(eye, target, up)[None, None, ...]  # shape [1, 1, 4, 4] 
print(viewmat)
viewmat = viewmat.to(device)    


# Simple intrinsics
WIDTH = 2124
HEIGHT = 2832
fx = fy = 2409.2728782415115
cx = 1062
cy = 1416
K = torch.tensor([[fx, 0, cx],
                  [0, fy, cy],
                  [0, 0, 1]], dtype=torch.float32)[None, None, ...]  # shape [1, 1, 3, 3]
K = K.to(device)
# -------------------------------
# Add batch dimension for GSplat
# -------------------------------
means = means.unsqueeze(0)
quats = quats.unsqueeze(0)
scales = scales.unsqueeze(0)
opacities = opacities.unsqueeze(0)
colors = colors.unsqueeze(0)

# -------------------------------
# Render image
# -------------------------------
render_colors, _, _  = rendering.rasterization(
    means=means,
    quats=quats,
    scales=scales,
    opacities=opacities,
    colors=colors,
    viewmats=viewmat,
    Ks=K,
    width=WIDTH,
    height=HEIGHT,
    # near_plane=NEAR,
    # far_plane=FAR
)

# -------------------------------
# Save rendered image
# -------------------------------
img = (render_colors[0].cpu().numpy() * 255).astype(np.uint8)
img = np.squeeze(img)  # removes all dimensions of size 1
# Check shape
print(img.shape)  # should be (height, width, 3)
Image.fromarray(img).save(os.path.join(os.path.expanduser("~/humanoid/output"), 'rendered_image.png'))
print(f"Saved rendered image to {os.path.join(os.path.expanduser('~/humanoid/output'), 'rendered_image.png')}")

tensor([[[[ 1.,  0.,  0.,  0.],
          [ 0.,  1.,  0.,  0.],
          [ 0.,  0.,  1., -5.],
          [ 0.,  0.,  0.,  1.]]]])
(2832, 2124, 3)
Saved rendered image to /home/astawu/humanoid/navlab-humanoid/notebooks/rendered_image.png


Feature matching light-glue

In [18]:
from lightglue import LightGlue, SuperPoint, DISK, SIFT, ALIKED, DoGHardNet
from lightglue.utils import load_image, rbd

In [19]:
# SuperPoint+LightGlue
extractor = SuperPoint(max_num_keypoints=2048).eval().cuda()  # load the extractor
matcher = LightGlue(features='superpoint').eval().cuda()  # load the matcher